# Imports

In [120]:
import matplotlib.pyplot as plt
import os
import random
import numpy as np
import tensorflow as tf
import xml.etree.ElementTree as ET

from PIL import Image, ImageDraw
from numpy import asarray
from bs4 import BeautifulSoup as bs

from object_detection.utils import config_util, visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [126]:
porcentagem_acerto = 0.2

# Funções de Controle de Imagem

In [122]:
def criarArquivo(contadores, path):
    path = path.replace("./results/multiple/imagens/", "./results/multiple/info/")
    path = path.replace(".jpg", ".txt")

    arquivo = open(path, "w")
    arquivo.write("WBC\n")
    arquivo.write(str(contadores[0]) + "\n")
    arquivo.write("RBC\n")
    arquivo.write(str(contadores[1]) + "\n")
    arquivo.write("Platelets\n")
    arquivo.write(str(contadores[2]) + "\n")
    arquivo.close()

In [127]:
# Converte a imagem em array
def convertImageToArray(path):
    image = Image.open(path)
    array = asarray(image)
    return array

# Imprime as detecções na imagem
def plotDetections(imagem, boxes, classes, scores, categoria, nome=None):
    imagemDeteccao = imagem.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        imagemDeteccao, boxes, classes, scores, categoria, use_normalized_coordinates=True, min_score_thresh=porcentagem_acerto)

    if nome:
        pontuacao = np.squeeze(scores)

        contador_wbc = 0
        contador_rbc = 0
        contador_platelets = 0
        print("start")
        for i in range(100):
            if scores is None or pontuacao[i] > porcentagem_acerto:
                print("batata", pontuacao[i], classes[i])
                if classes[i] == 1:
                    contador_wbc = contador_wbc + 1
                    print("wbc")
                elif classes[i] == 2:
                    contador_rbc = contador_rbc + 1
                elif classes[i] == 3:
                    contador_platelets = contador_platelets + 1

        contadores = [contador_wbc, contador_rbc, contador_platelets]

        plt.imsave(nome, imagemDeteccao)
        img = Image.open(nome)
        editor = ImageDraw.Draw(img)
        editor.text((10, 10), "teste", fill=(255, 255, 0))

        criarArquivo(contadores, nome)

        img.save(nome)
    else:
        plt.imshow(imagemDeteccao)


# Análise dos Dados

In [124]:
train_images_dir = './dataset-multiple/train/imagens/'
train_images = []

# Coleta as imagens, converte em array e salva em array geral
for elemento in os.listdir(train_images_dir):
    imagem = os.path.join(train_images_dir, elemento)
    train_images.append(convertImageToArray(imagem))

# Parâmetros para a plotagem
plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.labelsize'] = False
plt.rcParams['ytick.labelsize'] = False
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['figure.figsize'] = [14, 7]

# Plotagem da visualização teste
for indice in range(0, 6):
    plt.subplot(2, 3, indice+1)
    plt.imshow(train_images[indice])
plt.show()

KeyboardInterrupt: 

# Leitura das Coordenadas

In [ ]:
# Função para realizar a conversão das coordenadas do XML para o formato utilizado no plot
def calcularPosicao(lista):
    nome = lista[0]
    xmin = lista[1]
    ymin = lista[2]
    xmax = lista[3]
    ymax = lista[4]

    larguraImagem = 640
    alturaImagem = 480

    xmin = xmin / larguraImagem
    ymin = ymin / alturaImagem
    xmax = xmax / larguraImagem
    ymax = ymax / alturaImagem

    return [ymin, xmin, ymax, xmax]

In [ ]:
# Função para coletar as coordenadas do arquivo XML
def getCelula(arquivo): 
    celula = [] 
    arquivo = open(arquivo, "r")
    linhas = arquivo.readlines()
    celula.append(linhas[0].replace("\n", ""))
    celula.append(int(linhas[1].replace("\n", "")))
    celula.append(int(linhas[2].replace("\n", "")))
    celula.append(int(linhas[3].replace("\n", "")))
    celula.append(int(linhas[4].replace("\n", "")))
    arquivo.close()
                
    return celula

print(getCelula("dataset-multiple/train/coordenadas/cell_0.txt"))

In [ ]:
# Importação das coordenadas para um array
coordenadas_dir = 'dataset-multiple/train/coordenadas/'
coordenadas_info = []
titulos = []


def traduzir(nome):
    if (nome == "WBC"):
        return 1
    elif (nome == "RBC"):
        return 2
    elif (nome == "Platelets"):
        return 3
    else:
        return False


for arquivo in os.listdir(coordenadas_dir):
    celula = getCelula(coordenadas_dir + arquivo)
    coordenadas_info.append(
        np.array([calcularPosicao(celula)], dtype=np.float32))
    titulos.append(np.array([traduzir(celula[0])]))

print(len(train_images))
print(len(coordenadas_info))
print(len(titulos))

# Limitar número da amostra
limite = 500
train_images = train_images[:limite]
coordenadas_info = coordenadas_info[:limite]
titulos = titulos[:limite]

print(len(train_images))
print(len(coordenadas_info))
print(len(titulos))

# Preparando dados para treinamento

In [ ]:
# Definição da classe
wbc_id = 1
rbc_id = 2
platelets_id = 3
num_classes = 3

categorias = {wbc_id: {'id': wbc_id, 'name': 'wbc'},
              rbc_id: {'id': rbc_id, 'name': 'rbc'},
              platelets_id: {'id': platelets_id, 'name': 'platelets'}}

train_images_tensors = []
coordenadas_info_tensors = []
offset = 1
classes_tensors = []

# Função Zip = Tupla entre dois arryas (Imagens + Coordenadas)
for (train_image, coordenada, titulo) in zip(train_images, coordenadas_info, titulos):
    train_images_tensors.append(tf.expand_dims(tf.convert_to_tensor(train_image, dtype=tf.float32), axis=0))
    coordenadas_info_tensors.append(tf.convert_to_tensor(coordenada, dtype=tf.float32))
    indices = tf.convert_to_tensor(titulo - offset)
    classes_tensors.append(tf.one_hot(indices, num_classes))

print('Dados preparados')

# Visualizando imagens com coordenadas

In [ ]:
print(len(train_images))
print(len(coordenadas_info))
print(len(titulos))

In [ ]:
print(coordenadas_info[6])
print(titulos[6])

In [ ]:
import random

print()

fake_score = np.array([1.0], dtype=np.float32)

plt.figure(figsize=(10, 5))

teste = random.randint(0, len(train_images))

plotDetections(
        train_images[teste],
        coordenadas_info[teste],
        titulos[teste],
        fake_score, categorias)
plt.show()

In [ ]:
fake_score = np.array([1.0], dtype=np.float32)

plt.figure(figsize=(30, 15))
for indice in range(0, 6):
    plt.subplot(2, 3, indice+1)
    plotDetections(
        train_images[indice],
        coordenadas_info[indice],
        titulos[indice],
        fake_score, categorias)
plt.show()

# Criar modelo e trabalhar com checkpoints

In [ ]:
tf.keras.backend.clear_session()
print('Construindo modelo e restaurando pesos para fine-tuning...', flush=True)

# Parâmetros do modelo
num_classes = 3
pipeline_config = 'models/retinaNet50_640x640/pipeline.config'
checkpoint_path = 'models/retinaNet50_640x640/checkpoint/ckpt-0'

# Importação e definição
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True

# Construção do Modelo
model = model_builder.build(model_config=model_config, is_training=True)

fakebox_predictor = tf.compat.v2.train.Checkpoint(_base_tower_layers_for_heads=model._box_predictor._base_tower_layers_for_heads,
                                                  _box_prediction_head=model._box_predictor._box_prediction_head)
fake_model = tf.compat.v2.train.Checkpoint(
    _feature_extractor=model._feature_extractor, _box_predictor=fakebox_predictor)

checkpoint = tf.compat.v2.train.Checkpoint(model=fake_model)
checkpoint.restore(checkpoint_path).expect_partial()

image, shapes = model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = model.predict(image, shapes)
_ = model.postprocess(prediction_dict, shapes)
print('Pesos restaurados!')

# Loop de treinamento personalizado

In [ ]:
tf.keras.backend.set_learning_phase(True)

# Valores padrão
# batch_size = 4
# learning_rate = 0.01
# num_batches = 100

# Mehlores valores até agora
# batch_size = 24
# learning_rate = 0.02
# num_batches = 200

batch_size = 16
learning_rate = 0.02
num_batches = 200

# Variáveis do Treinamento
variaveis_treino = model.trainable_variables
ajuste_fino = []
prefixo_treino = ['WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
                  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']

for variavel in variaveis_treino:
    if any([variavel.name.startswith(prefix) for prefix in prefixo_treino]):
        ajuste_fino.append(variavel)


def passoTreino(model, otimizador, ajuste_fino):
    @tf.function
    def passo(imagem_tensors, boxes_list, classes_list):

        forma = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
        model.provide_groundtruth(
            groundtruth_boxes_list=boxes_list, groundtruth_classes_list=classes_list)

        with tf.GradientTape() as tape:
            prepro_images = tf.concat([model.preprocess(imagem_tensor)[
                                      0] for imagem_tensor in imagem_tensors], axis=0)
            predicao_dict = model.predict(prepro_images, forma)
            perdas_dict = model.loss(predicao_dict, forma)
            perda_total = perdas_dict['Loss/localization_loss'] + \
                perdas_dict['Loss/classification_loss']
            gradientes = tape.gradient(perda_total, ajuste_fino)
            otimizador.apply_gradients(zip(gradientes, ajuste_fino))
        return perda_total

    return passo


otimizador = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
passoTreino = passoTreino(model, otimizador, ajuste_fino)

print('Fine-tuning inicializado!', flush=True)
for indice in range(num_batches):
    chaves = list(range(len(train_images)))
    random.shuffle(chaves)
    exemplo_chaves = chaves[:batch_size]

    boxes_list = [coordenadas_info_tensors[chave] for chave in exemplo_chaves]
    classes_list = [classes_tensors[chave] for chave in exemplo_chaves]
    image_tensors = [train_images_tensors[chave] for chave in exemplo_chaves]
    perda_total = passoTreino(image_tensors, boxes_list, classes_list)

    if indice % 10 == 0:
        print('Batch ' + str(indice) + ' of ' + str(num_batches)
              + ', Loss = ' + str(perda_total.numpy()), flush=True)

print('Fine-tuning realizado!')

# Carregar imagens para teste e aplicar no modelo

In [128]:
test_image_dir = 'dataset-multiple/test/imagens/'
test_images = []
test_images_nomes = []

for elemento in os.listdir(test_image_dir):
    imagem = os.path.join(test_image_dir, elemento)
    test_images_nomes.append(elemento)
    test_images.append(np.expand_dims(convertImageToArray(imagem), axis=0))

@tf.function
def detectar(input_tensor):
    prepross_image, formas = model.preprocess(input_tensor)
    predicao_dict = model.predict(prepross_image, formas)
    return model.postprocess(predicao_dict, formas)

offset = 1
for i in range(len(test_images)):
    test_tensor = tf.convert_to_tensor(test_images[i], dtype=tf.float32)
    deteccoes = detectar(test_tensor)

    plotDetections(
        test_images[i][0],
        deteccoes['detection_boxes'][0].numpy(),
        deteccoes['detection_classes'][0].numpy().astype(np.uint32) + offset,
        deteccoes['detection_scores'][0].numpy(),
        categorias, nome="./results/multiple/imagens/" + test_images_nomes[i])


start
batata 0.48529452 2
batata 0.3752118 2
batata 0.24400361 2
batata 0.23688643 2
batata 0.23153098 2
batata 0.22381291 2
batata 0.22204635 2
batata 0.22181822 2
batata 0.21278319 1
wbc
batata 0.21277706 2
batata 0.20896031 2
start
batata 0.80508983 2
batata 0.6979515 2
batata 0.42900133 2
batata 0.40946576 2
batata 0.3825796 2
batata 0.32456967 2
batata 0.30694088 2
batata 0.2767248 2
batata 0.27448395 2
batata 0.27095017 2
batata 0.24966493 2
batata 0.214255 2
batata 0.2091106 1
wbc
batata 0.20811343 2
batata 0.20142554 2
batata 0.20038801 2
start
batata 0.3980038 2
batata 0.3233748 2
batata 0.29280436 2
batata 0.28517595 2
batata 0.27613717 2
batata 0.2394324 2
batata 0.21909817 2
batata 0.21713978 3
batata 0.21394722 2
batata 0.21387734 2
batata 0.20430535 2
start
batata 0.452676 2
batata 0.261087 2
batata 0.23065767 2
batata 0.22059478 1
wbc
batata 0.20961335 2
start
batata 0.96125853 2
batata 0.7526942 2
batata 0.441634 2
batata 0.36022633 2
batata 0.34197855 2
batata 0.305282